In [ ]:
import sys
sys.path.append('/home/azhuravl/work')

import stereoanyvideo.datasets.video_datasets as video_datasets

In [ ]:
import importlib
importlib.reload(video_datasets)

In [ ]:
dataset_driving = video_datasets.SequenceSceneFlowDatasetCamera(
    aug_params=None,
    root="/home/azhuravl/scratch/SceneFlow",
    dstype="frames_cleanpass",
    sample_len=59,
    things_test=False,
    add_things=False,
    add_monkaa=False,
    add_driving=True,
    split="test",
    stride=5,
)
len(dataset_driving)



In [ ]:
data_0 = dataset_tartanair[0]
data_1 = dataset_tartanair[1]


In [ ]:
import matplotlib.pyplot as plt
# plot 0th frame of the first sequence and 2nd sequence
plt.subplot(1, 2, 1)
plt.imshow(data_0['img'][50][0].permute(1, 2, 0).int().numpy())
# plt.imshow(data_0['img'][0][0].permute(1, 2, 0).int().numpy())
plt.title('Sequence 0, Frame 0')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(data_1['img'][0][0].permute(1, 2, 0).int().numpy())
plt.title('Sequence 1, Frame 0')
plt.axis('off')

plt.show()

In [ ]:
data_0['img'].shape

In [ ]:
import torch
from pytorch3d.utils.camera_conversions import opencv_from_cameras_projection

# If you have a PyTorch3D camera from your output_tensor
viewpoint = data_0["viewpoint"][0][0]  # First frame, left camera

# Convert to OpenCV format
# opencv_params = video_datasets.pytorch3d_to_opencv_camera_general(viewpoint, (540, 960))

R, t, K = opencv_from_cameras_projection(
    viewpoint,
    # image_size=torch.tensor([[540, 960]])
    # use the actual image shape
    image_size=torch.tensor([[data_0['img'].shape[-2], data_0['img'].shape[-1]]])
)
R = R[0]
t = t[0]
K = K[0]

# Access the parameters
# K = opencv_params['K']          # 3x3 intrinsic matrix
# R = opencv_params['R']          # 3x3 rotation matrix
# t = opencv_params['t']          # 3x1 translation vector
# calculate depth from disparity using torch

import torch
disp = data_0['disp'][0][0]
focal_length = K[0, 0]
baseline = 1
depth = -(focal_length * baseline) / disp

In [ ]:
# plot rgb and disparity for the first frame
import matplotlib.pyplot as plt
plt.figure(figsize=(12,6))
plt.subplot(1,3,1)
plt.imshow(data_0['img'][40][0].permute(1, 2, 0).int().numpy())
plt.title('RGB Frame 0')
plt.axis('off')
plt.subplot(1,3,2)
plt.imshow(data_0['disp'][40][0].permute(1, 2, 0).numpy(), cmap='plasma')
plt.title('Disparity Frame 0')
plt.axis('off')
plt.colorbar(fraction=0.03, pad=0.04)
plt.subplot(1,3,3)
plt.imshow(1 / depth.permute(1, 2, 0).numpy(), cmap='plasma')
plt.title('Depth Frame 0')
plt.axis('off')
plt.colorbar(fraction=0.03, pad=0.04)
plt.show()

## VKITTI 2

In [ ]:
import importlib
importlib.reload(video_datasets)

In [ ]:
train_sequences = [
    'abandonedfactory/Easy/P001',
    # 'abandonedfactory/Easy/P005', 
    # 'office/Easy/P001',
    # 'office/Easy/P002',
    # 'office2/Easy/P001'
]

dataset_tartanair = video_datasets.TartanAirDataset(
        aug_params=None,
        root="/home/azhuravl/scratch/tartanair",
        split="train",
        sample_len=59,
        only_first_n_samples=-1,
        sampling_stride=3,          # Starting frame stride (default 3)
        min_temporal_step=1,        # Minimum temporal step (default 1)  
        max_temporal_step=2,        # Maximum temporal step (default 6)
        train_sequences=train_sequences
    )

# dataset_vkitti = video_datasets.VKITTI2Dataset(
#     aug_params=None,
#     root="/home/azhuravl/scratch/vkitti2",
#     split="train",
#     sample_len=59,
#     only_first_n_samples=-1,
#     sampling_stride=8,          # Starting frame stride (default 3)
#     min_temporal_step=1,        # Minimum temporal step (default 1)  
#     max_temporal_step=1,        # Maximum temporal step (default 6)
# )

In [ ]:
data_0 = dataset_tartanair[0]

In [ ]:
data_0.keys()

In [ ]:
importlib.reload(collect_dataset)

In [ ]:
import torch

sys.path.append('/home/azhuravl/work/TrajectoryCrafter/notebooks/15_10_25_depth')

import collect_dataset

frames_tensor, depths, poses_tensor, K_tensor = collect_dataset.extract_video_data(
    data_0,
    baseline=0.25,
    image_size=frames_tensor.shape[-2:-1],
    )

In [ ]:
depths.shape

In [ ]:
poses_tensor = data_0['RTK']
depths = -data_0['depth2disp_scale'][0] / data_0['disp'][:,0,...] 
# depth = depth2disp_scale / disparity


In [ ]:
# depths

In [ ]:
# sys.path.append('/home/azhuravl/work/TrajectoryCrafter')

# import models.utils as utils

# warper_old = utils.Warper(device='cuda')

In [ ]:
from tqdm import tqdm

warped_images = []
masks = []
warped_depths = []

# poses_tensor_inv = torch.inverse(poses_tensor)

# flip x axis
# poses_tensor[:, 0:3, 3] = -poses_tensor[:, 0:3, 3]
# flip y axis
# poses_tensor[:, 1:3, 3] = -poses_tensor[:, 1:3, 3]
# flip z axis
# poses_tensor[:, 2:3, 3] = -poses_tensor[:, 2:3, 3]



for i in tqdm(range(10, frames_tensor.shape[0])):
    
    # transformation_1 = torch.inverse(poses_tensor[i:i+1])
    # transformation_2 = torch.inverse(poses_tensor[10:11])
    
    transformation_1 = poses_tensor[i:i+1].clone()
    transformation_2 = poses_tensor[10:11].clone()
    
    # multiply translation by 100
    # transformation_1[:, 0:3, 3] = transformation_1[:, 0:3, 3] * 100
    # transformation_2[:, 0:3, 3] = transformation_2[:, 0:3, 3] * 100
    
    # transformation_1 = torch.inverse(transformation_1)
    # transformation_2 = torch.inverse(transformation_2)
    
    # print(transformation_1)
        
    warped_frame2, mask2, warped_depth2, flow12 = warper_old.forward_warp(
        frame1=frames_tensor[i:i+1],
        mask1=None,
        depth1=depths[i:i+1],
        transformation1=transformation_1,
        transformation2=transformation_2,
        intrinsic1=K_tensor[i:i+1],
        intrinsic2=K_tensor[i:i+1],
        mask=False,
        twice=False,
    )
    # print(warped_frame2[0])
    warped_images.append(warped_frame2)
    masks.append(mask2)
    warped_depths.append(warped_depth2)
    
    # print(warped_frame2.shape, mask2.shape, warped_depth2.shape)

In [ ]:
import numpy as np

# plot warped image j, mask j, warped depth j
j = 0
k = 30

frame = frames_tensor[j+10].cpu().permute(1, 2, 0).numpy() * 0.5 + 0.5
warped_image = warped_images[k][0].cpu().permute(1, 2, 0).numpy() * 0.5 + 0.5
target_frame = frames_tensor[k+10].cpu().permute(1, 2, 0).numpy() * 0.5 + 0.5

plt.figure(figsize=(18,6))
plt.subplot(1,3,1)
plt.imshow(frame)
plt.title('Source Frame')
plt.axis('off')

plt.subplot(1,3,2)
plt.imshow(warped_image)
plt.title('Warped Image to Frame {}'.format(j))
plt.axis('off')

plt.subplot(1,3,3)
plt.imshow(target_frame)
plt.title('Target Frame {}'.format(j))
plt.axis('off')

# plt.subplot(1,3,1)
# plt.imshow(depths[10+j].cpu().permute(1, 2, 0).numpy(), cmap='plasma')
# plt.title('Warped Image to Frame {}'.format(j))
# plt.axis('off')
# plt.colorbar()

# plt.subplot(1,3,2)
# plt.imshow((warped_depths[j][0].cpu().permute(1, 2, 0).numpy() + 1e-2), cmap='plasma')
# plt.title('Warped Image to Frame {}'.format(j))
# plt.axis('off')
# plt.colorbar()

# plt.subplot(1,3,3)
# plt.imshow(masks[j][0].cpu().permute(1, 2, 0).numpy() * 0.5 + 0.5, cmap='gray')
# plt.title('Mask')
# plt.axis('off')
# plt.show()


In [ ]:
from models.utils import save_video
import os

def make_dimensions_even(tensor):
    """Pad tensor to make height and width even numbers"""
    _, h, w, c = tensor.shape
    pad_h = h % 2
    pad_w = w % 2
    
    if pad_h > 0 or pad_w > 0:
        # Pad bottom and right if needed
        tensor = torch.nn.functional.pad(tensor, (0, 0, 0, pad_w, 0, pad_h))
    
    return tensor


cond_video = (torch.cat(warped_images) + 1.0) / 2.0  # [T, 3, H, W] in [0,1]
cond_video_padded = make_dimensions_even(
    cond_video.permute(0, 2, 3, 1)
)

save_video(
    cond_video_padded,
    '/home/azhuravl/work/TrajectoryCrafter/notebooks/22_10_25_scaling_up/warped_video.mp4',
    fps=10,
)
# --- save inputs for visualization ---

input_video_padded = make_dimensions_even(
    (frames_tensor[10:].permute(0, 2, 3, 1) + 1.0) / 2.0
)

save_video(
    input_video_padded,
    '/home/azhuravl/work/TrajectoryCrafter/notebooks/22_10_25_scaling_up/input_video.mp4',
    fps=10,
)

warped_depths_tensor = torch.cat(warped_depths)
# Apply before saving
warped_depths_padded = make_dimensions_even(
    (warped_depths_tensor.permute(0, 2, 3, 1).repeat(1, 1, 1, 3)) / warped_depths_tensor.max()
)
save_video(
    warped_depths_padded,
    '/home/azhuravl/work/TrajectoryCrafter/notebooks/22_10_25_scaling_up/warped_depths.mp4',
    fps=10,
)

depths_padded = make_dimensions_even(
    (depths.permute(0, 2, 3, 1).repeat(1, 1, 1, 3)) / depths.max()
)
save_video(
    depths_padded,
    '/home/azhuravl/work/TrajectoryCrafter/notebooks/22_10_25_scaling_up/input_depths.mp4',
    fps=10,
)

In [ ]:
import sys
sys.path.append('/home/azhuravl/work/TrajectoryCrafter/notebooks/06_10_25_vggt')

import warper_point_cloud
warper = warper_point_cloud.GlobalPointCloudWarper(device='cuda')


In [ ]:
from tqdm import tqdm

pc_list = []
color_list = []

with torch.no_grad():
    for j in tqdm(range(frames_tensor.shape[0])):
        i = j
        points, colors, _ = warper.create_pointcloud_from_image(
            frames_tensor[i:i+1],
            None,
            depths[i:i+1],
            # torch.inverse(poses_tensor[i:i+1].to('cuda') @ transform_mat).to('cuda'),
            torch.inverse(poses_tensor[i:i+1].to('cuda')),
            K_tensor[i:i+1],
            1,
        )
        pc_list.append(points)
        color_list.append(colors)

In [ ]:
import viser
import numpy as np

# Start viser server
server = viser.ViserServer()
import os

# print slurm node name

node_name=os.environ.get('SLURM_NODELIST', 'localhost')

print(f'http://{node_name}:{server.get_port()}')


def add_points(
    server,
    points: np.ndarray,  # (N, 3)
    colors: np.ndarray,  # (N, 3)
    name: str,
):
    # ensure colors are in [0, 1]
    if colors.min() < -0.1:
        colors = (colors + 1.0) / 2.0
    
    server.scene.add_point_cloud(
        name=name,
        points=points,
        colors=colors,
        point_size=0.01
    )

def add_camera(
    server,
    pose: np.ndarray,  # (4, 4)
    name: str,
    color: tuple = (0.2, 0.8, 0.2),
):
    pose = np.linalg.inv(pose)
    
    position = pose[:3, 3]
    rotation_matrix = pose[:3, :3]
    
    # Convert rotation to quaternion
    wxyz = viser.transforms.SO3.from_matrix(rotation_matrix).wxyz
    
    server.scene.add_camera_frustum(
        name,
        fov=60, aspect=4/3, scale=0.1,
        position=position, wxyz=wxyz,
        color=color
    )
    
# Add this after creating your server and adding point clouds
@server.on_client_connect
def _(client: viser.ClientHandle) -> None:
    yaw_slider = client.gui.add_slider("Camera Yaw", min=-180, max=180, step=1, initial_value=0)
    
    @yaw_slider.on_update
    def _(_):
        angle_rad = np.deg2rad(yaw_slider.value)
        radius = 8.0
        
        position = np.array([
            radius * np.sin(angle_rad),
            0,  # height
            radius * np.cos(angle_rad)
        ])
        
        client.camera.position = position
        client.camera.look_at = np.array([0, 0, 0])
        client.camera.up_direction = np.array([0, -1, 0])


In [ ]:
poses_tensor

In [ ]:
server.scene.reset()

# add original point cloud #0, and all cameras from first segment

j_list = [0, 15, 30, 45]

for j in j_list:    
    add_points(
        server,
        pc_list[j].cpu().numpy(),
        color_list[j].cpu().numpy(),
        name=f'input_pc_{j}'
    )
    
for j in range(frames_tensor.shape[0]):
    add_camera(
        server,
        torch.inverse(poses_tensor[j]).cpu().numpy(),
        name=f'input_cam_{j:02d}',
        color=(0.2, 0.8, 0.2)
    )
   

In [ ]:
sys.path.append('/home/azhuravl/work/TrajectoryCrafter/notebooks/06_10_25_vggt')
from parsing import get_parser
import utils_autoregressive as utils_ar
from datetime import datetime
import os
import copy


sys.argv = [
    "",
    "--video_path", "/home/azhuravl/nobackup/DAVIS_testing/trainval/monkaa.mp4",
    "--n_splits", "4",
    "--overlap_frames", "0",
    "--radius", "0",
    "--mode", "gradual",
]

parser = get_parser()
opts_base = parser.parse_args()

timestamp = datetime.now().strftime("%Y%m%d_%H%M")
video_basename = os.path.splitext(os.path.basename(opts_base.video_path))[0]

# Setup
opts_base.weight_dtype = torch.bfloat16
opts_base.exp_name = f"{video_basename}_{timestamp}_autoregressive"
opts_base.save_dir = os.path.join(opts_base.out_dir, opts_base.exp_name)

# Create TrajCrafterVisualization instance for autoregressive generation
radius = opts_base.radius

variants = [
    ("right_90", [0, 90, radius, 0, 0]),
]

pose = [90, 0, 0, 0, 1]
name = f"{pose[0]}_{pose[1]}_{pose[2]}_{pose[3]}_{pose[4]}"

opts = copy.deepcopy(opts_base)
opts.exp_name = f"{video_basename}_{timestamp}_{name}_auto_s{opts_base.n_splits}"
opts.save_dir = os.path.join(opts.out_dir, opts.exp_name)
opts.camera = "target"
opts.target_pose = pose
opts.traj_txt = 'test/trajs/loop2.txt'

# Make directories
os.makedirs(opts.save_dir, exist_ok=True)

In [ ]:
trajcrafter = utils_ar.TrajCrafterAutoregressive(opts)

In [ ]:
import numpy as np

# frames_tensor = (
    # torch.from_numpy(frames_np).permute(0, 3, 1, 2).to(opts.device) * 2.0 - 1.0
  # 49 576 1024 3 -> 49 3 576 1024, [-1,1]
# reverse this to get frames in numpy
frames_np = ((frames_tensor.cpu().permute(0, 2, 3, 1).numpy() + 1.0) / 2.0).astype(np.float32)

trajcrafter.prompt = trajcrafter.get_caption(opts, frames_np[opts.video_length // 2])
print(trajcrafter.prompt)

In [ ]:
_, segment_dir = utils_ar.sample_diffusion(
    trajcrafter,
    frames_tensor[10:],
    warped_images,
    frames_tensor[:10],
    masks,
    opts,
)

## TartanAir

In [ ]:
import importlib
importlib.reload(video_datasets)

In [ ]:
train_sequences = [
    'abandonedfactory/Easy/P001',
    # 'abandonedfactory/Easy/P005', 
    # 'office/Easy/P001',
    # 'office/Easy/P002',
    # 'office2/Easy/P001'
]

dataset_tartanair = video_datasets.TartanAirDataset(
        aug_params=None,
        root="/home/azhuravl/scratch/tartanair",
        split="train",
        sample_len=59,
        only_first_n_samples=-1,
        sampling_stride=3,          # Starting frame stride (default 3)
        min_temporal_step=1,        # Minimum temporal step (default 1)  
        max_temporal_step=2,        # Maximum temporal step (default 6)
        train_sequences=train_sequences
    )

In [ ]:
data_0 = dataset_tartanair[10]

In [ ]:
data_0

In [ ]:
data_0 = dataset_tartanair[0]
data_1 = dataset_tartanair[1]


In [ ]:
# TODO: fix TartanAir cameras